In [244]:
import transformers
import pandas as pd
import sklearn
import torch

In [245]:
torch.manual_seed(1)
device = torch.device('cpu')

In [246]:
model_name = "KB/bert-base-swedish-cased"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at KB/bert-base-swedi

In [247]:
class QuestionDataset(torch.utils.data.Dataset):
    def __init__(self, questions, labels, tokenizer):
        self.questions = questions
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, item):
        question = str(self.questions[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=64,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
        ).to(device)

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": label,
        }

In [248]:
data = pd.read_csv('dataset.csv')

# Divide the data into training, validation and test
training_data, testing_data = sklearn.model_selection.train_test_split(data, test_size=0.2, random_state = 1)
training_data, validation_data = sklearn.model_selection.train_test_split(training_data, test_size = 0.2, random_state = 1)

print("training data: ", training_data.shape)
print("validation data: ", validation_data.shape)
print("testing data: ", testing_data.shape)

training data:  (5120, 2)
validation data:  (1280, 2)
testing data:  (1600, 2)


In [249]:
# Load datasets into QuestionDataset objects
training_dataset = QuestionDataset(questions=training_data["question"].tolist(), labels=training_data["label"].tolist(), tokenizer=tokenizer)
validation_dataset = QuestionDataset(questions=validation_data["question"].tolist(), labels=validation_data["label"].tolist(), tokenizer=tokenizer)
testing_dataset = QuestionDataset(questions=testing_data["question"].tolist(), labels=testing_data["label"].tolist(), tokenizer=tokenizer)

In [250]:
def collate_fn(samples):
    input_ids = []
    attention_masks = []
    labels = []

    for sample in samples:
        input_ids.append(sample['input_ids'])
        attention_masks.append(sample['attention_mask'])
        labels.append(sample['label'])

    # Padding
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True)
    attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True)
    labels = torch.tensor(labels)

    return input_ids, attention_masks, labels

In [251]:
# Load the pre-trained model
model_name = "KB/bert-base-swedish-cased"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

# Set up the optimizer and the loss function
optimizer = torch.optim.Adam(model.parameters(), lr = 2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Load your dataset into a DataLoader
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

# Fine-tune the model on your dataset
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    batch_count = 0
    for batch in train_loader:
        # Show progress in percentage
        batch_count += 1
        print(f"\r{batch_count/len(train_loader)*100:.2f}%", end="")

        # Load the batch into the device (GPU)
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Clear the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask = attention_mask)
        logits = outputs.logits

        # Compute the loss
        loss = loss_fn(logits, labels)

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

    # Evaluate the model on the validation set
    val_loader = torch.utils.data.DataLoader(validation_dataset, batch_size = 32)
    model.eval()
    val_preds = []
    val_labels = []
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask = attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        val_preds.extend(preds.tolist())
        val_labels.extend(labels.tolist())
    val_acc = sklearn.metrics.accuracy_score(val_labels, val_preds)
    print(f"Validation accuracy: {val_acc:.4f}")

Some weights of the model checkpoint at KB/bert-base-swedish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at KB/bert-base-swedi

Epoch 1/3
10.94%